In [1]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import numpy as np
# import gmaps

# Import API key
# from api_keys import geoapify_key
# from api_keys import gmaps_key

# gmaps.configure(api_key={gmaps_key})

In [2]:
headers = {
    "User-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}

In [3]:
# data to scrape
job_title = []
company_name = []
short_description = []
job_location = []

job_counter = 0

In [4]:
job_field = ['construction']
page_scrapes = ['', '?page=2', '?page=3','?page=4','?page=5', '?page=6', '?page=7', '?page=8', '?page=9', '?page=10'] #'?page=6', '?page=7'

In [5]:
# for x in job_field:

for page in page_scrapes:

        url = 'https://www.seek.com.au/construction-jobs' + page
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, features='html.parser')
        jobs_on_page = soup('article')
            
        for i in range(len(jobs_on_page)):

            job_counter += 1 
            
            job_title.append(soup('article')[i]['aria-label'])
            short_description.append(soup('article')[i].find('span', {'data-automation':'jobShortDescription'}).text)
            job_location.append(soup('article')[i].find('a', {'data-automation':'jobLocation'}).text)
            
            print('Jobs scraped: %d' % job_counter, end='\r')
            
            try:
                company_name.append(soup('article')[i].find('a', {'data-automation':'jobCompany'}).text)
            except:
                company_name.append(np.nan)

In [6]:
print(len(company_name))
print(len(job_title))
print(len(short_description))
print(len(job_location))

220
220
220
220


In [7]:
seek_data = {'job_title': job_title,
            'company_name': company_name,
            'short_description': short_description,
            'job_location': job_location,
             }

In [8]:
seek_data_df = pd.DataFrame(seek_data)
seek_data_df.reset_index(inplace=True)
seek_data_df.rename(columns= {'index':'job_id'}, inplace=True)
seek_data_df.tail(150)

,job_id,job_title,company_name,short_description,job_location
70,70,Asphalt Crew Labourers/Operators Positions Ava...,J & E Asphalt & Civil Pty Ltd,Asphalt Crew workers required to join our Asph...,Riverstone
71,71,Contracts Administrator,Rork Projects,Join a well established team. Diverse pipeline...,Sydney NSW
72,72,Cadet Project Supervisor,Global 360,Do you have a Construction / Building Trade ba...,Doncaster
73,73,Road Worker,Chandler Macleod,Road Worker required for major road works proj...,Tamworth
74,74,Junior Project Manager,Atom Resources,Reputable luxury residential firm seeking a dr...,Sydney NSW
...,...,...,...,...,...
215,215,Contract Administrator,Woods & Co,Tier 2 builder is seeking a Contract Administr...,Melbourne VIC
216,216,HR Truck Drivers / Labourers URGENT,Red Rock Recruitment,Our client is an industry leader in the develo...,Loganholme
217,217,Site Supervisors - John Hunter Health and Inno...,Multiplex Constructions,Excellent opportunity for experienced Site Sup...,Newcastle
218,218,Supervisor,"CPB Contractors WA, SA & NT",Be a part of something bigger. Forge your care...,Alkimos


In [9]:
# Word Cloud Data

full_count = (seek_data_df['job_title'].str.split(expand=True)
              .stack())
full_count_df = pd.DataFrame(full_count)
full_count_df = full_count_df[full_count_df[0].str.len() > 2]
full_count_df

0
0   0    Construction
    1      Supervisor
1   0      Structural
    1        Engineer
2   0          Junior
...               ...
217 7      Innovation
    8        Precinct
218 0      Supervisor
219 0        Projects
    1  Superintendant

[727 rows x 1 columns]

In [10]:
# count, tally and describe words in job_title

construction_count = (seek_data_df['job_title'].str.split(expand=True)
              .stack()
              .value_counts()
              .rename_axis('word')
              .reset_index(name='count'))

In [11]:

construction_count = construction_count[construction_count['word'].str.len() > 4]
construction_count

,word,count
0,Project,72
1,Manager,71
3,Construction,38
4,Administrator,37
6,Junior,31
...,...,...
226,Coordinator/,1
227,"Construction,",1
228,PROJECT,1
229,operations,1


In [12]:
construction_count.head(45)

,word,count
0,Project,72
1,Manager,71
3,Construction,38
4,Administrator,37
6,Junior,31
7,Supervisor,23
8,Contracts,20
9,Coordinator,17
11,Contract,14
13,Cadet,13


In [13]:
# create df for location counts
construction_locations = seek_data_df['job_location'].value_counts()

construction_locations_df = pd.DataFrame(construction_locations)


In [14]:

construction_locations_df.head(20)

,job_location
Sydney NSW,30
Melbourne VIC,18
Adelaide SA,6
Canberra ACT,6
Newcastle,6
Brisbane QLD,5
West Melbourne,4
Shepparton,4
Ballarat,4
Fyshwick,3


In [15]:
construction_locations_df.reset_index(inplace=True)

construction_locations_df

,index,job_location
0,Sydney NSW,30
1,Melbourne VIC,18
2,Adelaide SA,6
3,Canberra ACT,6
4,Newcastle,6
...,...,...
113,Sydney Olympic Park,1
114,Alkimos,1
115,Bacchus Marsh,1
116,Queanbeyan,1


In [18]:
# construction_locations_df.rename(columns= {'index':'job_location', 
#                                            'job_location':'count'}, inplace=True)

construction_locations_df.head(20)

,job_location,count
0,Sydney NSW,30
1,Melbourne VIC,18
2,Adelaide SA,6
3,Canberra ACT,6
4,Newcastle,6
5,Brisbane QLD,5
6,West Melbourne,4
7,Shepparton,4
8,Ballarat,4
9,Fyshwick,3


In [17]:
# MOVE TO TOP BEFORE SUBMITTING

from api_keys import postgres_p
from sqlalchemy import create_engine
from sqlalchemy import inspect

ModuleNotFoundError: No module named 'api_keys'

In [ ]:
protocol = 'postgresql'
username = 'postgres'
password = postgres_p

host = 'localhost'

port = 5432

database_name = 'Project_3'

rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

insp.get_table_names()

In [ ]:
# load to database

# Job_ID Table

job_id_df.to_sql(name='job_id', con=engine, if_exists='append', index=False)

#Scraped job table
construction_df.to_sql(name='construction_scrapes', con=engine, if_exists='append', index=False)

# Word count
construction_count_df.to_sql(name='construction_words', con=engine, if_exists='append', index=False)

# Location
construction_locations_df.to_sql(name='construction_location_count', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from construction_location_count', con=engine).head(15)